In [1]:
import pandas as pd
import itertools, os
from collections import OrderedDict

#### External fluxes data

In [2]:
df_ex = pd.read_excel('./raw_data_files/Elsemman_Nielsen_Teusink_2022_exflux.xlsx',
                      sheet_name='TI', skiprows=14, index_col=0)

In [4]:
exr_dict = OrderedDict({'qetoh': 'EX_etoh_e', 'qglyc': 'EX_glyc_e',
            'q pyruvate': 'EX_pyr_e', 'qace': 'EX_ac_e', 'q succinate': 'EX_succ_e',
            'q citrate': 'EX_cit_e', 'qCO2': 'EX_co2_e'})

cond_dict = {'batchGlcCyHx1': ('EX_glc__D_e', ['R5 0.15', 'R6 0.15']),
             'batchGlcCyHx2': ('EX_glc__D_e', ['R3 0.25_2', 'R4 0.25_2']),
             'batchGlcCyHx3': ('EX_glc__D_e', ['R5 0.33', 'R6 0.33']),
             'batchGlcCyHx4': ('EX_glc__D_e', ['R5 0.37', 'R6 0.37'])}

fname_prefix = 'Elsemman2022'
for cond,keys in cond_dict.items():
    sid,ridx = keys
    fpath = './' + fname_prefix + '_' + cond + '/'
    if os.path.isdir(fpath) == False:
        os.makedirs(fpath)
        
    ptext = []
    # Growth rate
    mu = round(df_ex.loc[ridx, 'mu (/h)'].mean(),4)
    ptext += ['* Growth rate', '$setGlobal mu ' + str(mu),
              "v.fx('BIOSYN-BIODIL') = %mu% * %nscale%;", ""]
    
    # Carbon substrate uptake and oxygenation
    upt = -round(df_ex.loc[ridx, 'qs mmol/gDW/h'].mean(),4)
    o2 = -round(df_ex.loc[ridx, 'qO2'].mean(),4)
    
    ptext += ['* Substrate and oxygenation',
              "v.fx('RXN-" + sid + "_REV-SPONT') = " + str(upt) + " * %nscale%;",
              "v.fx('RXN-" + sid + "_FWD-SPONT') = 0 * %nscale%;", "",
              "v.fx('RXN-EX_o2_e_REV-SPONT') = " + str(o2) + " * %nscale%;",
              "v.fx('RXN-EX_o2_e_FWD-SPONT') = 0 * %nscale%;", ""]
    
    # Secretions
    ptext += ['* Secretions']
    for col,secid in exr_dict.items():
        val = round(df_ex.loc[ridx, col].mean(), 4)
        if val > 0:
            ptext += ["v.lo('RXN-" + secid + "_FWD-SPONT') = " + str(val) + " * %nscale%;",
                      "v.fx('RXN-" + secid + "_REV-SPONT') = 0 * %nscale%;", ""]
    
    with open(fpath + 'phenotype_frompy.txt', 'w') as f:
        f.write('\n'.join(ptext))